### **Sales Analysis**

Total sales in the last quarter

In [ ]:
SELECT 
    SUM(TotalDue) AS TotalSalesLastQuarter
FROM SalesLT.SalesOrderHeader
WHERE OrderDate >= DATEADD(QUARTER, -1, GETDATE());

Average number of products sold per transaction

In [ ]:
SELECT 
    AVG(OrderQty) AS AvgItemsPerOrder
FROM SalesLT.SalesOrderDetail;

Top 5 customers with the highest total purchase value

In [ ]:
SELECT TOP 5 
    c.CustomerID,
    c.FirstName,
    c.LastName,
    SUM(soh.TotalDue) AS TotalSpent
FROM SalesLT.Customer c
JOIN SalesLT.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
GROUP BY c.CustomerID, c.FirstName, c.LastName
ORDER BY TotalSpent DESC;

Month-over-month sales comparison

In [4]:
SELECT 
    FORMAT(OrderDate, 'yyyy-MM') AS Month,
    SUM(TotalDue) AS MonthlySales
FROM SalesLT.SalesOrderHeader
GROUP BY FORMAT(OrderDate, 'yyyy-MM')
ORDER BY Month;

(1 row affected)

Total execution time: 00:00:00.450

Month,MonthlySales
2008-06,"956303,5949"
